In [ ]:
import os
# 【硬性要求】加载环境变量，确保读取 OPENAI_API_KEY
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# 【规范更新】旧版本：from langchain_openai.chat_models import ChatOpenAI
# 现代写法：统一直接从 langchain_openai 顶层导入
from langchain_openai import ChatOpenAI

# 引入 Redis 消息历史和历史记录包装器
from langchain_redis import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# ==========================================
# 1. 搭建基础链 (Base Chain) 
# ==========================================
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}. Respond in 20 words or fewer."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

base_chain = prompt | model

# ==========================================
# 2. 定义记忆检索策略 (Memory Retrieval Strategy)
# ==========================================
REDIS_URL = "redis://localhost:6379/0"
# 先启动docker 还要在终端运行以下命令启动 Redis 容器：
# docker run -d -p 6379:6379 redis
# 这个函数会被 RunnableWithMessageHistory 在每次链调用时动态调用




# 【科研级保姆注释】
# 这是一个工厂函数。LangChain 每次调用链时，都会动态调用这个函数。
# 它根据传入的 session_id，返回一个指向该会话的 Redis 连接对象。
def get_message_history(session_id: str) -> RedisChatMessageHistory:
    # 显式指定参数名 session_id 和 redis_url
    return RedisChatMessageHistory(
        session_id=session_id, 
        redis_url=REDIS_URL
    )

# ==========================================
# 3. 组装“带有记忆的链” (Chain with Memory)
# ==========================================
# 使用 RunnableWithMessageHistory 将基础链和记忆工厂包装在一起
redis_chain = RunnableWithMessageHistory(
    base_chain,
    get_message_history, # 传入上面定义的工厂函数句柄 (Function Reference)
    input_messages_key="input",   # 告诉它哪个变量是用户的当前输入
    history_messages_key="history" # 告诉它哪个变量用来塞入历史记录
)

# ==========================================
# 4. 执行并发多租户测试 (Multi-tenant Testing)
# ==========================================
if __name__ == "__main__":
    print("=== 开始测试：数学线程 (Session: math-thread1) ===")
    
    # 回忆上一节课的知识！session_id 是通过 configurable 动态注入的
    config_math = {"configurable": {"session_id": "math-thread1"}}
    
    resp1 = redis_chain.invoke(
        {"ability": "math", "input": "What does cosine mean?"},
        config=config_math
    )
    print(f"User: What does cosine mean?\nAI: {resp1.content}\n")
    
    resp2 = redis_chain.invoke(
        {"ability": "math", "input": "Tell me more!"}, # 第二次调用，没有提供前置上下文
        config=config_math
    )
    print(f"User: Tell me more!\nAI: {resp2.content}\n")
    
    print("=== 开始测试：物理线程隔离 (Session: phy-thread1) ===")
    
    # 切换到全新的 Session ID，验证记忆是否隔离
    config_phy = {"configurable": {"session_id": "phy-thread1"}}
    
    resp3 = redis_chain.invoke(
        {"ability": "physics", "input": "What is the theory of relativity?"},
        config=config_phy
    )
    print(f"User: What is the theory of relativity?\nAI: {resp3.content}\n")
    
    resp4 = redis_chain.invoke(
        {"ability": "physics", "input": "Tell me more!"}, 
        config=config_phy
    )
    # AI 会继续解释相对论，而不是数学的余弦定理，这证明多租户隔离成功！
    print(f"User: Tell me more!\nAI: {resp4.content}\n")

=== 开始测试：数学线程 (Session: math-thread1) ===
User: What does cosine mean?
AI: Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.

User: Tell me more!
AI: Cosine is abbreviated as cos. It's used to calculate angles and distances in trigonometry and has a periodic nature.

=== 开始测试：物理线程隔离 (Session: phy-thread1) ===
User: What is the theory of relativity?
AI: Theory explaining how gravity affects time and space, proposed by Einstein. Includes special and general relativity.

User: Tell me more!
AI: Special relativity deals with motion at constant speed, while general relativity includes gravity's effect on spacetime.

